In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import Flatten, Dropout, Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
print(tf.__version__)
print(keras.__version__)

In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split

In [ ]:
columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
df = pd.read_csv('/home/carlos/Documents/Base/Dataset/merged21.csv', header = None, names = columns)
df

In [ ]:
df['x-axis'] = df['x-axis'].astype('int64')
df['y-axis'] = df['y-axis'].astype('int64')
df['z-axis'] = df['z-axis'].astype('int64')
df.info()

In [ ]:
df['activity'].value_counts().plot(kind='bar', title='Training examples by activity type')

In [ ]:
WINDOW_SIZE = 200
OUTPUT_LABELS = 5
step = 20
LABELS = ['Jogging', 'Running', 'Jumping', 'Fall', 'Standing']

# load data
segments = []
labels = []
for i in range(0, len(df) - WINDOW_SIZE, step):
    xa = df['x-axis'].values[i: i + WINDOW_SIZE]
    ya = df['y-axis'].values[i: i + WINDOW_SIZE]
    za = df['z-axis'].values[i: i + WINDOW_SIZE]
    label = np.zeros(OUTPUT_LABELS, dtype=np.uint8)
    label[LABELS.index(stats.mode(df['activity'][i: i + WINDOW_SIZE])[0][0])] = 1

    segments.append([xa, ya, za])
    labels.append(label)

In [ ]:
reshaped_segments = np.asarray(segments, dtype = np.uint8)
reshaped_segments = np.transpose(reshaped_segments, [0, 2, 1])
labels = np.asarray(labels)

In [ ]:
reshaped_segments = np.reshape(reshaped_segments, [-1, 10, 20 ,3])

In [ ]:
RANDOM_SEED=123
X_train, X_test, y_train, y_test = train_test_split(reshaped_segments, labels, test_size=0.10, random_state=RANDOM_SEED)
X_train, X_valid, y_train, y_valid= train_test_split(X_train, y_train, test_size=0.20, random_state=RANDOM_SEED)

print('segments shape: ', reshaped_segments.shape)
print(reshaped_segments.shape[0], 'training samples')
print('labels shape: ', labels.shape)

print('train data', X_train.shape)
print('valid data', X_valid.shape)
print('test data', X_test.shape)

In [ ]:
DROP_OUT = 0.2
OPTIMIZER_LR = 0.001
DENSE_UNIT_1 = 200
DENSE_UNIT_2 = 100

model = Sequential()
model.add(ZeroPadding2D(padding=((1, 1), (1, 1)), input_shape=(10, 20, 3)))
model.add(Conv2D(32, (3,3), strides=(1, 1), padding='valid', activation='relu', input_shape=(10, 20, 3)))

model.add(ZeroPadding2D(padding=((1, 1), (1, 1))))
model.add(Conv2D(64, (3, 3), strides=(1, 1),  padding='valid', activation='relu'))    
model.add(MaxPooling2D(pool_size=(2, 2), padding=("same")))
    
model.add(Flatten())

model.add(Dense(DENSE_UNIT_1, activation="relu"))
model.add(Dropout(DROP_OUT))

model.add(Dense(DENSE_UNIT_2, activation="relu"))
model.add(Dropout(DROP_OUT))

model.add(Dense(OUTPUT_LABELS, activation="softmax"))
adam = Adam(lr = OPTIMIZER_LR, decay=1e-6)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=8, batch_size=512, verbose=2)

In [ ]:
model.summary()

In [ ]:
def plot_learningCurve(history, epochs):
  # Plot training & validation accuracy values
  epoch_range = range(1, epochs+1)
  plt.plot(epoch_range, history.history['accuracy'])
  plt.plot(epoch_range, history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(epoch_range, history.history['loss'])
  plt.plot(epoch_range, history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

plot_learningCurve(history, 8)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import seaborn as sns


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

pred_y = model.predict(X_test.astype(np.float32))
pred_y_classes = np.argmax(pred_y, axis = 1) 
tue_y= np.argmax(y_test, axis = 1) 
confusion_mtx = confusion_matrix(tue_y, pred_y_classes) 
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mtx, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d", cmap='Blues')
plt.title("Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
for i in range(0, X_test.shape[0]):
  pilImg = Image.fromarray(np.uint8(X_test[i]))
  pilImg.save("/home/carlos/Documents/Base/Images/" + LABELS[np.argmax(y_test[i])] + "/" + str(i) + ".jpg")

In [ ]:
model.save('/home/carlos/Documents/Base/Model/last.hdf5')

In [ ]:
!tflite_convert  --output_file="/home/carlos/Documents/Base/Model/last.tflite" --keras_model_file="/home/carlos/Documents/Base/Model/last.hdf5"

In [ ]:
%cd /home/carlos/Documents/Base/Maix_Toolbox
!./ncc/ncc -i tflite -o k210model --dataset "/home/carlos/Documents/Base/Images/" "/home/carlos/Documents/Base/Model/last.tflite" "/home/carlos/Documents/Base/Model/last.kmodel"